https://github.com/hwchase17/langchain/blob/master/docs/modules/chains/examples/multi_retrieval_qa_router.ipynb

In [11]:
from langchain.chains.router import MultiRetrievalQAChain
from langchain.llms import OpenAI

import util_funs.globalsettings as gs

In [25]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

vector_store_pdf = Chroma(
    collection_name='pdf_files',
    embedding_function=OpenAIEmbeddings(),
    persist_directory=gs.the_folders.DIR_PERSIST
    )
pdf_retriever = vector_store_pdf.as_retriever()

vector_store_csv_spain = Chroma(
    collection_name='csv_files_spain',
    embedding_function=OpenAIEmbeddings(),
    persist_directory=gs.the_folders.DIR_PERSIST
    )
csv_spain_retriever = vector_store_csv_spain.as_retriever()

vector_store_csv_ue = Chroma(
    collection_name='csv_files_ue',
    embedding_function=OpenAIEmbeddings(),
    persist_directory=gs.the_folders.DIR_PERSIST
    )
csv_ue_retriever = vector_store_csv_ue.as_retriever()

Using embedded DuckDB with persistence: data will be stored in: C:\Users\migue\Documents\PROYECTOS DATA SCIENCE\langchain\PYTHON\util_funs\..\..\data\chroma
Using embedded DuckDB with persistence: data will be stored in: C:\Users\migue\Documents\PROYECTOS DATA SCIENCE\langchain\PYTHON\util_funs\..\..\data\chroma
Using embedded DuckDB with persistence: data will be stored in: C:\Users\migue\Documents\PROYECTOS DATA SCIENCE\langchain\PYTHON\util_funs\..\..\data\chroma


In [33]:
retriever_infos = [
    {
        "name": "situacion economia", 
        "description": "Good for answering questions about economy", 
        "retriever": pdf_retriever
    },
    {
        "name": "datos paro españa", 
        "description": "Good for answering questions about unemployement in spain", 
        "retriever": csv_spain_retriever
    },
    {
        "name": "datos paro UE", 
        "description": "Good for answering questions about unemployement in ue", 
        "retriever": csv_ue_retriever
    }
]

In [34]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [41]:
chain = MultiRetrievalQAChain.from_retrievers(OpenAI(), retriever_infos, verbose=True)

In [42]:
print(chain.run("Qué impacto tuvo la guerra de Ucrania en la economia durante 2022?"))



> Entering new MultiRetrievalQAChain chain...
situacion economia: {'query': 'Qué impacto tuvo la guerra de Ucrania en la economía durante 2022?'}
> Finished chain.
 La guerra de Ucrania tuvo un impacto negativo en la economía durante 2022, ya que provocó una escalada de los precios de materias primas, especialmente energéticas, una subida de tipos de interés más rápida de lo anticipado, y una reducción temporal del suministro de algunos bienes. Esto provocó una desaceleración del crecimiento económico en el primer trimestre del año, aunque el segundo trimestre ha visto una aceleración gracias al buen comportamiento del consumo de servicios.


In [37]:
print(chain.run("cuales son las tasas de paro total en españa en T12023?"))



> Entering new MultiRetrievalQAChain chain...
datos paro españa: {'query': 'cuales son las tasas de paro total en españa en T12023?'}
> Finished chain.
 La tasa de paro total en España en el periodo 2023T1 es 11.57 para hombres, 15.13 para mujeres y 12.13 para ambos sexos de 25 a 54 años.


In [38]:
print(chain.run("cual es la tasa de paro total nacional en españa del 2023T1"))



> Entering new MultiRetrievalQAChain chain...
datos paro españa: {'query': 'cual es la tasa de paro total nacional en españa del 2023T1'}
> Finished chain.
 La tasa de paro total nacional en España del 2023T1 es 13.26.


In [19]:
pdf_retriever.get_relevant_documents("Qué impacto tuvo la guerra de Ucrania en la economia durante 2022?")

[Document(page_content='La evolución de la inflación durante los últimos trimestres provoca la revisión al alza de la previsión del deflactor del consumo privado para 2022 y 2023. En línea con lo esperado en el Programa de Estabilidad y con lo señalado por otros organismos, la guerra de Rusia en Ucrania está resultando en un entorno global más adverso y está exacerbando las tensiones globales de precios en los mercados del gas y del petróleo, así como de otras materias primas. La guerra afecta a la economía española tanto en los precios como en la actividad, a través de varios canales, entre los que destaca el alza de los costes energéticos que se está traduciendo en unos niveles de inflación elevados que retraen el consumo. El incremento de los precios está siendo más acusado y está durando más de lo inicialmente previsto, lo que ha llevado a los principales\n\n22\n\nInforme de situación de la economía española 2022\n\norganismos como el BCE y Banco de España a revisar al alza sus pre

In [23]:
csv_retriever.get_relevant_documents("cual es la tasa de paro total nacional en españa del 2023T1?")

[Document(page_content='ï»¿zona: Spain\ntasa paro: 12.9\nmes: 2023-01', metadata={'source': 'C:\\Users\\migue\\Documents\\PROYECTOS DATA SCIENCE\\langchain\\PYTHON\\util_funs\\..\\..\\data\\pdf_csv_files\\tasa_paro_mensual_por_pais_ue.csv', 'row': 11}),
 Document(page_content='ï»¿zona: Spain\ntasa paro: 12.9\nmes: 2023-01', metadata={'source': 'C:\\Users\\migue\\Documents\\PROYECTOS DATA SCIENCE\\langchain\\PYTHON\\util_funs\\..\\..\\data\\pdf_csv_files\\tasa_paro_mensual_por_pais_ue.csv', 'row': 11}),
 Document(page_content='ï»¿zona: Spain\ntasa paro: 12.9\nmes: 2023-02', metadata={'source': 'C:\\Users\\migue\\Documents\\PROYECTOS DATA SCIENCE\\langchain\\PYTHON\\util_funs\\..\\..\\data\\pdf_csv_files\\tasa_paro_mensual_por_pais_ue.csv', 'row': 45}),
 Document(page_content='ï»¿zona: Spain\ntasa paro: 12.9\nmes: 2023-02', metadata={'source': 'C:\\Users\\migue\\Documents\\PROYECTOS DATA SCIENCE\\langchain\\PYTHON\\util_funs\\..\\..\\data\\pdf_csv_files\\tasa_paro_mensual_por_pais_ue.csv